In [ ]:
import numpy as np
from decimal import *
from scipy.special import comb

getcontext().prec = 128

In [ ]:
def rdp2dp(rdp, bad_event, alpha):
    return rdp + 1.0/(alpha-1) * (np.log(1.0/bad_event) + (alpha-1)*np.log(1-1.0/alpha) - np.log(alpha))

In [ ]:
from autodp import rdp_bank, rdp_acct, dp_acct,privacy_calibrator

def get_eps_bounded(q, sigma, bad_event, iters):
    func_gaussian= lambda x: rdp_bank.RDP_gaussian({'sigma': sigma }, x)

    # declare the moment accountants
    acct1 = rdp_acct.anaRDPacct()

    acct1.compose_subsampled_mechanism(func_gaussian, q, coeff=iters, improved_bound_flag=True)
    return acct1.get_eps(bad_event)

In [ ]:
def get_rdp_tau(alpha, sigma, c, k):
    sum_ = Decimal(0.0)
    
    p_r = [Decimal(comb(k, i, exact=True)) * Decimal(2.0/c)**Decimal(i) * Decimal(1-2.0/c)**Decimal(k-i) for i in range(0, k+1)]
    
    for i in range(0, k+1):
        sum_ +=  p_r[i]*Decimal(np.e)**(Decimal((alpha-1)*alpha*i**2)/Decimal(2*sigma**2))
    rdp = sum_.ln() / Decimal(alpha-1)
    return float(rdp)

def get_sampled_row_rdp(alpha, sigma, m, p_list, c):
    sum_ = Decimal(0.0)
    
    for k in range(0, m+1):
        tau = get_rdp_tau(alpha, sigma, c, k)
        sum_ +=  p_list[k]*Decimal(np.e)**( Decimal((alpha-1) * tau) )
        
    rdp = sum_.ln() / Decimal(alpha-1)
    
    return float(rdp)

def search_sampled_row_budget(sigma, m, p_list, bad_event, c, iters=1):
    min_dp = 1e5
    for alpha in list(range(2, 101)):
        rdp = iters * get_sampled_row_rdp(alpha, sigma, m, p_list, c)
        dp = rdp2dp(rdp, bad_event, alpha)
        min_dp = min(min_dp, dp)
    return min_dp

### Calibrating Number of Iterations

In [ ]:
bad_event = 1e-4
sigma = 10
m = 32
c = 10
q = 0.05

p_list_p = [Decimal(comb(m, k, exact=True)) * Decimal(q)**Decimal(k) * Decimal(1-q)**Decimal(m-k) for k in range(0, m+1)]
sampled_row_rdp_list = []
for alpha in list(range(2, 101)):
    sampled_row_rdp_list.append(get_sampled_row_rdp(alpha, sigma, m, p_list_p, c))

In [ ]:
def search_sampled_row_budget_fast(iters=1):
    min_dp = 1e5
    for alpha in list(range(2, 101)):
        rdp = iters * sampled_row_rdp_list[alpha-2]
        dp = rdp2dp(rdp, bad_event, alpha)
        min_dp = min(min_dp, dp)
    return min_dp

In [ ]:
def get_T(target_gamma):
    gamma = search_sampled_row_budget_fast(1)
    t = 1
    while gamma < target_gamma:
        t += 1
        gamma = search_sampled_row_budget_fast(t)
    return t

In [ ]:
T_list = []
for tg in range(2, 9, 1):
    T_list.append(get_T(tg))

In [ ]:
# total number of iterations for different privacy budgets in [2,8]
print(T_list)

### Calibrating Number of Iterations using Baseline Solution

In [ ]:
def get_T_naive(target_gamma):
    t_max = 100
    t_min = 1
    
    mid = int((t_max+t_min)/2)
    
    while t_max - t_min > 1:
        eps = m*get_eps_bounded(q, sigma, bad_event, mid)
        ratio = (np.e**(eps/m)-1)/(np.e**eps-1)
        tmp_g = m*get_eps_bounded(q, sigma, ratio*bad_event, mid)
        
        if tmp_g > target_gamma:
            t_max = mid
        else:
            t_min = mid
        mid = int((t_max+t_min)/2)
    return t_max

In [ ]:
T_list = []
for tar_g in range(2, 9, 1):
    T_list.append(get_T_naive(tar_g))

In [ ]:
# number of iterations calibrated by naive group privacy solution
print(T_list)